In [131]:
# Imports
import numpy as np
import pandas as pd
import datetime

In [132]:
#data = pd.concat([pd.read_csv('Data/2009.csv'), pd.read_csv('Data/2010.csv'), 
 #                 pd.read_csv('Data/2011.csv'), pd.read_csv('Data/2012.csv'), 
  #                pd.read_csv('Data/2013.csv'), pd.read_csv('Data/2014.csv'), 
   #               pd.read_csv('Data/2015.csv'), pd.read_csv('Data/2016.csv'), 
    #              pd.read_csv('Data/2017.csv'), pd.read_csv('Data/2018.csv')])

# Read 2018 data only
data = pd.read_csv('Data/2010.csv')

In [133]:
# Convert string to DateTime and set as index
data.FL_DATE = pd.to_datetime(data.FL_DATE, infer_datetime_format=True)
data.set_index('FL_DATE', inplace=True)
data.drop(columns='Unnamed: 27', axis=1, inplace=True) # Drop last weird column


In [134]:
# Drop NaN values in the variables we will use in our baseline model
data = data[data['TAXI_OUT'].notna()]
data = data[data['ORIGIN'].notna()]
data = data[data['DEST'].notna()]


In [135]:
# Take first 6 months
df = data.loc['2010-01-01':'2010-06-30']


In [136]:
# Renaming airline codes to company names
# Source: https://en.wikipedia.org/wiki/List_of_airlines_of_the_United_States

df['OP_CARRIER'].replace({
    'UA':'United Airlines',
    'AS':'Alaska Airlines',
    '9E':'Endeavor Air',
    'B6':'JetBlue Airways',
    'EV':'ExpressJet',
    'F9':'Frontier Airlines',
    'G4':'Allegiant Air',
    'HA':'Hawaiian Airlines',
    'MQ':'Envoy Air',
    'NK':'Spirit Airlines',
    'OH':'PSA Airlines',
    'OO':'SkyWest Airlines',
    'VX':'Virgin America',
    'WN':'Southwest Airlines',
    'YV':'Mesa Airline',
    'YX':'Republic Airways',
    'AA':'American Airlines',
    'DL':'Delta Airlines'
},inplace=True)

/Users/frederikhartmann/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [137]:
df = df[df.ORIGIN == 'JFK']

In [138]:
delays = df[['DEP_DELAY', 'DEP_TIME']]
delays['DEP_TIME'] = delays['DEP_TIME'].astype(int)

preps = []
for i in range(len(delays['DEP_TIME'])):
    # Zero fill values
    dep_time_val = str(delays['DEP_TIME'].iloc[i]).zfill(4)
    # If flight at 24:00, set that as 00:00
    if dep_time_val == str(2400):
        dep_time_act = datetime.datetime.strptime('0000','%H%M').strftime('%H:%M')
    else:
        dep_time_act = datetime.datetime.strptime(dep_time_val,'%H%M').strftime('%H:%M')
    

    # append
    preps.append(dep_time_act)

# Drop and add corrected times
df.drop(columns=['DEP_TIME'])
df['DEP_TIME'] = preps
df['DEP_TIME'] = pd.to_datetime(df['DEP_TIME'], format='%H:%M') # Convert to datetime

# Convert FL time to column
df = df.reset_index(level=0)

/var/folders/fs/10_w6dn131d7nmw8myr18l6r0000gn/T/ipykernel_52207/2530428706.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delays['DEP_TIME'] = delays['DEP_TIME'].astype(int)


In [139]:
comb_date = []
# Loop through all to combine dates and time
for i in range(len(df['DEP_TIME'])):

    # Get date and time
    date = datetime.datetime.date(df['FL_DATE'].iloc[i])
    time = datetime.datetime.time(df['DEP_TIME'].iloc[i])

    # Get combined as a string
    comb = datetime.datetime.combine(date, time).strftime('%Y-%m-%d %H:%M:%S')

    #append
    comb_date.append(comb)


In [140]:
# Add column, and remove DEP_TIME and FL_DATE
df['DATE_TIME'] = pd.to_datetime(comb_date)
# Get dep time as time only
df['DEP_TIME'] = pd.to_datetime(df['DEP_TIME'], format='%H:%M').dt.time # Convert to datetime


In [141]:
# Move it to the front of the data frame
dates = df.pop('DATE_TIME')
df.insert(0, 'DATE_TIME', dates)


### WEATHER DATA

In [142]:
# Load weather data
df_weather_all = pd.read_csv('Data/Weather.csv')
# Select only relevant features
df_weather = df_weather_all[["NAME","DATE","HLY-TEMP-NORMAL","HLY-WIND-AVGSPD"]]
# Select only JFK airport and from January 1st to July 1st
df_weather= df_weather[(df_weather["NAME"] == "JFK INTERNATIONAL AIRPORT, NY US") & (df_weather["DATE"] < '07-01T00:00:00')]

In [143]:
# Change the format of DATE to datetime
df_weather["DATE"] = pd.to_datetime(df_weather["DATE"],format='%m-%dT%H:%M:%S',infer_datetime_format='%d-%m-%H')
# Change the format to month - day - hour
df_weather["DATE"] = df_weather["DATE"].dt.strftime("%m-%d-%H")
# Change NAME to just JFK
df_weather["NAME"] = 'JFK'
# Reset index
df_weather = df_weather.reset_index().drop('index',axis=1)
df_weather

# Select interesting features
df = df[["FL_DATE","OP_CARRIER","DEP_TIME","TAXI_OUT", "DEP_DELAY"]]

# Now we must merge df_weather and df
df["TEMP"] = 0
df["WIND"] = 0

for i in range(len(df)):
    # Filter out the month, day and hour 
    hour = df["DEP_TIME"][i].strftime("%H")
    monthday = df["FL_DATE"][i].strftime("%m-%d")
    
    if monthday+'-'+hour == '01-01-00':
        continue
    else:
        # Find temp in df_weather corresponding to month, day and hour
        df.iloc[i,-2] = df_weather[df_weather["DATE"] == monthday+'-'+hour].iloc[0,2]
        # Find wind in df_weather corresponding to month, day and hour
        df.iloc[i,-1] = df_weather[df_weather["DATE"] == monthday+'-'+hour].iloc[0,3]
        
# The hour of january 1st is not included in weather data. Therefore we drop the flights occurring in that time
df = df[df.TEMP > 0]

In [145]:
# Average delays
y_mean = df.groupby('FL_DATE', as_index=False)['DEP_DELAY'].mean()
y_std = df.groupby('FL_DATE', as_index=False)['DEP_DELAY'].std()

#df.set_index('FL_DATE', inplace=True)

In [150]:
# Impose delay from previous day
df["DELAY_PREV_AVG"] = 0
df["DELAY_PREV_STD"] = 0

# Set index as date
# Loop
for i in range(len(y_mean)):
    # If we are at day 1, continue as we dont have a day previous to that
    if y_mean['FL_DATE'][i].strftime('%d') == '01':
        continue

    # Take date
    date = y_mean['FL_DATE'][i].strftime('%Y-%m-%d')
    
    df.loc[df.FL_DATE == date, 'DELAY_PREV_AVG'] = y_mean['DEP_DELAY'][i-1]
    df.loc[df.FL_DATE == date, 'DELAY_PREV_STD'] = y_std['DEP_DELAY'][i-1]
    

In [154]:
df.head()

,FL_DATE,DEP_TIME,TAXI_OUT,DEP_DELAY,TEMP,WIND,DELAY_PREV_AVG,DELAY_PREV_STD,OP_CARRIER_American Airlines,OP_CARRIER_Delta Airlines,OP_CARRIER_Endeavor Air,OP_CARRIER_Envoy Air,OP_CARRIER_ExpressJet,OP_CARRIER_JetBlue Airways,OP_CARRIER_Mesa Airline,OP_CARRIER_PSA Airlines,OP_CARRIER_US,OP_CARRIER_United Airlines
0,2010-01-01,22:36:00,18.0,121.0,34.7,12.4,0.0,0.0,0,0,0,1,0,0,0,0,0,0
1,2010-01-01,16:22:00,21.0,7.0,38.1,12.9,0.0,0.0,0,0,0,1,0,0,0,0,0,0
2,2010-01-01,22:38:00,23.0,3.0,34.7,12.4,0.0,0.0,0,0,0,1,0,0,0,0,0,0
3,2010-01-01,10:24:00,23.0,-1.0,35.4,13.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0
4,2010-01-01,18:27:00,31.0,-3.0,36.5,12.8,0.0,0.0,0,0,0,1,0,0,0,0,0,0


In [151]:
# One hot encode the carriers
y = pd.get_dummies(df.OP_CARRIER, prefix='OP_CARRIER')
# Drop column B as it is now encoded
df.drop(columns='OP_CARRIER', inplace=True)
# Join the encoded df
df = df.join(y)


In [152]:
# Compare shape to see effect of preprocessing
print('Shape before:',data.shape,'\n Shape after:',df.shape)

Shape before: (6338168, 26) 
 Shape after: (54180, 18)


In [153]:
# Save as new dataframe
df.to_csv('Data/data_pre.csv')
df_weather.to_csv('Data/weather_pre.csv')